In [1]:
import os
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import MeCab

os.environ['MECABRC'] = "/etc/mecabrc"

## 企業URLの生成

In [9]:
urls = []
for i in range(0,1990):
    urls.append("https://webdb.tsjiba.or.jp/main/company/" + "{:08}".format(i) + "/show")

## descriptionタグの取得

In [10]:
descriptions = {}
for url in tqdm(urls):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.find('title').get_text()
    if title != "燕三条ものづくり企業ナビ":
        description = soup.find('meta', {'name' : 'description'})['content']
        descriptions[url] = description
    time.sleep(3)

100%|██████████| 1990/1990 [1:50:49<00:00,  3.34s/it]


## データの保存

In [14]:
import pickle
with open('data/tsubame_discriptions.pickle', 'wb') as f:
    pickle.dump(descriptions, f)

In [15]:
len(descriptions)

755

## データの読み込み

In [2]:
import pickle
import csv
import itertools
with open('data/tsubame_discriptions.pickle', 'rb') as f:
    descriptions = pickle.load(f)
with open("./data/base_wrods.pickle", "rb") as f:
    base_words = pickle.load(f)
base_word_list = [base_word[0] for base_word in base_words]
with open("./data/including_base_words.pickle", "rb") as f:
    including_base_words = pickle.load(f)
including_base_word_list = list(itertools.chain.from_iterable(including_base_words.values()))

## アノテーションデータの作成

In [3]:
# 文章をスペース区切りに変換
def get_text_splited_by_space(sentence):
    tagger = MeCab.Tagger()
    parsed_sentence = tagger.parse(sentence)
    
    word_list = parsed_sentence.split("\n")
    text_splited_by_space = ""
    for word in word_list:
        organized_info = {}
        
        word = word.split("\t")
        if word[0] == "EOS":
            break
        text_splited_by_space += word[0] + " " 
        
    return text_splited_by_space[:-1]

In [4]:
anotation_data = []
for key in descriptions.keys():
    labels = ""
    if descriptions[key] == "":
        continue
    text_splited_by_space = get_text_splited_by_space(descriptions[key].replace("\r", "").replace(",", "，").replace("　", ""))
    for word in text_splited_by_space.split(" "):
        if word in base_word_list:
            labels += "1 "
        else:
            labels += "0 "
    anotation_data.append([key, text_splited_by_space, labels[:-1]])

In [5]:
anotation_data[:5]

[['https://webdb.tsjiba.or.jp/main/company/00000002/show',
  'フェザー 安全 剃刀 （株） ・ ミキモト 装身具 （株） の OEM 生産 を し て いる 。 鋳造 技術 を 生かし 高級 金銀 製品 の 製造 ・ 研磨 仕上 や 極小 品 の 製造 を 得意 と する 。',
  '0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0'],
 ['https://webdb.tsjiba.or.jp/main/company/00000003/show',
  '創業 ９ ０ 年 以上 の 老舗 企業 として 、 高 精度 を 求め られる 機械 加工 部品 、 発電 、 化学 プラント で 使用 さ れる 高水準 の ボルト ナット の 製造 を 得意 と し て おり ます 。 機械 加工 部品 で は 、 電力 、 プラント 、 ...',
  '0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0'],
 ['https://webdb.tsjiba.or.jp/main/company/00000004/show',
  '小型 精密 部品 の プレス 加工 ・ 高 精度 の 金 型 製造 。 材料 仕入れ から 完成 品 の 梱包 ・ 出荷 まで 一貫 体制 。 特に SUS 材 について は 研磨 仕上げ ・ スポット 溶接 ・ アッ センブリ まで 行う 。',
  '0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0'],
 ['https://webdb.tsjiba.or.jp/main/company/00000005/show',
  'SUS 材 の 深 絞り 加工 を 得意 と し て い ます 。 金 型 に SKD 材 を 使う こと で 超 硬 合金 ・ フェロチック・ 表面 硬化 処理 等 に 比べ

## データの整形と保存

In [6]:
with open('data/anotated_tsubame_discriptions_for_train.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(anotation_data)